In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("API_KEY")
print(api_key)

In [31]:
def get_comments(youtube, **kwargs):
    comments = []
    results = youtube.commentThreads().list(**kwargs).execute()

    while results:
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)

        # check if there are more comments
        if 'nextPageToken' in results:
            kwargs['pageToken'] = results['nextPageToken']
            results = youtube.commentThreads().list(**kwargs).execute()
        else:
            break

    return comments


def comm(video_id, api_key):
    # Disable OAuthlib's HTTPs verification when running locally.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    youtube = googleapiclient.discovery.build(
        "youtube", "v3", developerKey=api_key)

    comments = get_comments(youtube, part="snippet",
                            videoId=video_id, textFormat="plainText")
    return comments


def get_video_comments(video_id):
    return comm(video_id, api_key)




In [32]:
def videoid(url):
    if "?v=" in url:
        video_id = url.split("=")[1].split("&")[0]
        return video_id
    else:
        video_id = url.split("/")[3].split("?")[0]
        return video_id

In [33]:
video_id = videoid(
    "https://youtu.be/OlPO8f-AAJk?si=ebQ8SWDaGRIj2F-b")
video_id

'OlPO8f-AAJk'

In [22]:
url = "https://youtu.be/RTYqgoKQWds?si=DP83nIc7TnuOrCMq"

# Split the URL by "=" and take the second part, then split by "&" and take the first part
video_id = url.split("/")[3].split("?")[0]

print("YouTube Video ID:", video_id)


YouTube Video ID: RTYqgoKQWds


In [ ]:
comm = get_video_comments("OlPO8f-AAJk")
comm

In [ ]:
comm = get_video_comments(video_id)
comm

In [19]:
len(comm)

216

In [4]:
leng = len(comm)

In [5]:
leng

1628

In [ ]:
comm[230]

In [15]:
len(comm[230])

39

In [20]:
num = 0
for i in range(leng):
    n = len(comm[i])
    num += n
print(num/leng)
print(num)

143.03931515535828
225573


In [ ]:
import streamlit as st
from dotenv import load_dotenv
import googleapiclient.errors
import googleapiclient.discovery
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


tokenizer = AutoTokenizer.from_pretrained(
    'nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained(
    'nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
num = 0
lis = []
for i, comment in enumerate(comm, 1):
    #tokens = tokenizer.encode(comment, return_tensors='pt')
    tokens = tokenizer.encode(comment, return_tensors='pt', max_length=512)

    result = model(tokens)
    sentiment = int(torch.argmax(result.logits)) + 1
    num += sentiment
    lis.append(f"Comment {i}: {comment} (Sentiment: {sentiment})")
print(num/len(comm))


for line in lis:
    print(line)